# Why Spark?
**1. Speed:** Run programs up to 100x faster than Hadoop MapReduce in memory, or 10x faster on disk.

**2. Ease of Use**

**3. Generality:** Combine SQL, streaming, and complex analytics.

**4. Runs Everywhere:** Spark runs on Hadoop, Mesos, standalone, or in the cloud. It can access diverse data sources including HDFS, Cassandra, HBase, and S3.

Spark powers a stack of libraries including SQL and DataFrames, MLlib for machine learning, GraphX, and Spark Streaming. You can combine these libraries seamlessly in the same application.
![](./image/stack.png)


## Demo Code in this Section

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark SQL basic example")\
    .config("spark.some.config.option", "some-value").getOrCreate()
# when set to true, the first line of files name columns and are not included in data.
# automatically infer column types. It requires one extra pass over the data and is false by default.


df = spark.sparkContext.parallelize([(1, 2, 3, 'a b c'),
                                     (4, 5, 6, 'd e f'),
                                     (7, 8, 9, 'g h i')]).toDF(['col1', 'col2', 'col3', 'col4'])

df.show()


+----+----+----+-----+
|col1|col2|col3| col4|
+----+----+----+-----+
|   1|   2|   3|a b c|
|   4|   5|   6|d e f|
|   7|   8|   9|g h i|
+----+----+----+-----+



In [5]:
type(df), df.columns, df.describe(), df.printSchema()

root
 |-- col1: long (nullable = true)
 |-- col2: long (nullable = true)
 |-- col3: long (nullable = true)
 |-- col4: string (nullable = true)



(pyspark.sql.dataframe.DataFrame,
 ['col1', 'col2', 'col3', 'col4'],
 DataFrame[summary: string, col1: string, col2: string, col3: string, col4: string],
 None)

In [11]:
# set up  SparkSession
#  inferSchema -> Infer schema will automatically guess the data types for each field
df = spark.read.format('csv').options( inferschema='true').load(
    "./data/Advertising.csv", header=True)

df.show(5)
df.printSchema()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



# Spark Components

![](./image/spark-components.png)

1. Spark Driver

    * separate process to execute user applications
    
    
    * creates SparkContext to schedule jobs execution and negotiate with cluster manager

2. Executors

    * run tasks scheduled by driver
    * store computation results in memory, on disk or off-heap
    * interact with storage systems

3. Cluster Manager

    * Mesos
    * YARN
    * Spark Standalone

**Spark Driver contains more components responsible for translation of user code into actual jobs executed on cluster:**



represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and broadcast variables on that cluster

+ DAGScheduler

computes a DAG of stages for each job and submits them to TaskScheduler determines preferred locations for tasks (based on cache status or shuffle files locations) and finds minimum schedule to run the jobs


![](./image/spark-components1.png)


+ TaskScheduler

responsible for sending tasks to the cluster, running them, retrying if there are failures, and mitigating stragglers

+ SchedulerBackend

backend interface for scheduling systems that allows plugging in different implementations(Mesos, YARN, Standalone, local)

+ BlockManager

provides interfaces for putting and retrieving blocks both locally and remotely into various stores (memory, disk, and off-heap)

# How Spark Works?

Spark has a small code base and the system is divided in various layers. Each layer has some responsibilities. The layers are independent of each other.


The first layer is the interpreter, Spark uses a Scala interpreter, with some modifications. As you enter your code in spark console (creating RDD’s and applying operators), Spark creates a operator graph. 

When the user runs an action (like collect), the Graph is submitted to a DAG Scheduler. The DAG scheduler divides operator graph into (map and reduce) stages. A stage is comprised of tasks based on partitions of the input data. 

The DAG scheduler pipelines operators together to optimize the graph. For e.g. Many map operators can be scheduled in a single stage. This optimization is key to Sparks performance. The final result of a DAG scheduler is a set of stages. 

The stages are passed on to the Task Scheduler. The task scheduler launches tasks via cluster manager. (Spark Standalone/Yarn/Mesos). The task scheduler doesn’t know about dependencies among stages.

![](./image/work_flow.png)

# Programming with RDDs

## what is RDD?

RDD represents `Resilient Distributed Dataset`. 

An RDD in Spark is simply an immutable distributed collection of objects sets. Each RDD is split into multiple partitions (similar pattern with smaller sets), which may be computed on different nodes of the cluster.

## How to create RDD

Usually, there are two popular ways to create the RDDs:

1. **loading an external dataset**, or 

2. **distributing a set of collection of objects**. 

The following examples show some simplest ways to create RDDs by using parallelize() fucntion which takes an already existing collection in your program and pass the same to the Spark Context.

### By using parallelize( ) function

In [12]:
from pyspark.sql import SparkSession




df = spark.sparkContext.parallelize([
    (1, 2, 3, 'a b c'),
    (4, 5, 6, 'd e f'),
    (7, 8, 9, 'g h i')]).toDF(['col1', 'col2', 'col3', 'col4'])
df.show()


+----+----+----+-----+
|col1|col2|col3| col4|
+----+----+----+-----+
|   1|   2|   3|a b c|
|   4|   5|   6|d e f|
|   7|   8|   9|g h i|
+----+----+----+-----+



In [13]:
type(df)


pyspark.sql.dataframe.DataFrame

In [14]:
df.collect()


[Row(col1=1, col2=2, col3=3, col4='a b c'),
 Row(col1=4, col2=5, col3=6, col4='d e f'),
 Row(col1=7, col2=8, col3=9, col4='g h i')]

parallelize() distribute a local python collection to form an RDD. Common built-in python collections include dist, list, tuple or set.

In [15]:
from pyspark.sql import SparkSession



myData = spark.sparkContext.parallelize(
    [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)])
myData.collect()


[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]

In [17]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# from a list of tuple
list_t = [('cat', 'dog', 'fish'), ('orange', 'apple')]
rdd = sc.parallelize(list_t)
rdd.collect()


[('cat', 'dog', 'fish'), ('orange', 'apple')]

In [18]:
# from a set
s = {'cat', 'dog', 'fish', 'cat', 'dog', 'dog'}
rdd = sc.parallelize(s)
rdd.collect()


['dog', 'fish', 'cat']

In [19]:
# from a dict
d = {
    'a': 100,
    'b': 200,
    'c': 300
}
rdd = sc.parallelize(d)
rdd.collect()


['a', 'b', 'c']

In [20]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("Python Spark create RDD example") \
#     .config("spark.some.config.option", "some-value") \
#     .getOrCreate()
myData = spark.sparkContext.parallelize([(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)])
myData.collect()


[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]

### By using createDataFrame( ) function


In [21]:
from pyspark.sql import SparkSession

# spark = SparkSession \
#     .builder \
#     .appName("Python Spark create RDD example") \
#     .config("spark.some.config.option", "some-value") \
#     .getOrCreate()

Employee = spark.createDataFrame([
    ('1', 'Joe',   '70000', '1'),
    ('2', 'Henry', '80000', '2'),
    ('3', 'Sam',   '60000', '2'),
    ('4', 'Max',   '90000', '1')],
    ['Id', 'Name', 'Sallary', 'DepartmentId'] # this is columns name
)
Employee.show()


+---+-----+-------+------------+
| Id| Name|Sallary|DepartmentId|
+---+-----+-------+------------+
|  1|  Joe|  70000|           1|
|  2|Henry|  80000|           2|
|  3|  Sam|  60000|           2|
|  4|  Max|  90000|           1|
+---+-----+-------+------------+



In [22]:
from pyspark.sql import Row
rdd = sc.parallelize([
    Row(x=[1, 2, 3], y=['a', 'b', 'c']),
    Row(x=[4, 5, 6], y=['e', 'f', 'g'])
])
rdd.collect()


[Row(x=[1, 2, 3], y=['a', 'b', 'c']), Row(x=[4, 5, 6], y=['e', 'f', 'g'])]

In [23]:
df = spark.createDataFrame(rdd)
df.show()

KeyboardInterrupt: 

### By using read and load functions
#### Read dataset from .csv file


In [20]:


df = spark.read.format('csv').\
    options(header='true',
            inferschema='true').\
    load("./data/Advertising.csv")

df.show()
df.printSchema()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
| 66.1|  5.8|     24.2|  8.6|
|214.7| 24.0|      4.0| 17.4|
| 23.8| 35.1|     65.9|  9.2|
| 97.5|  7.6|      7.2|  9.7|
|204.1| 32.9|     46.0| 19.0|
|195.4| 47.7|     52.9| 22.4|
| 67.8| 36.6|    114.0| 12.5|
|281.4| 39.6|     55.8| 24.4|
| 69.2| 20.5|     18.3| 11.3|
|147.3| 23.9|     19.1| 14.6|
+-----+-----+---------+-----+
only showing top 20 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



#### textFile()
The textFile() function reads a text file and returns it as an RDD of strings. Usually, you will need to apply some map functions to transform each elements of the RDD to some data structure/type that is suitable for data analysis.

When using textFile(), each line of the text file becomes an element in the resulting RDD.

Examples:

In [36]:
# read a csv file
rdd = sc.textFile('./data/mtcars.csv')


In [39]:
# read a txt file
rdd = sc.textFile('./data/twitter.txt')
# rdd.take(2)


#### Read dataset from HDFS

HDFS : HDFS has been designed to be easily portable from one platform to another. This facilitates widespread adoption of HDFS as a platform of choice for a large set of applications.



In [40]:
# from pyspark.conf import SparkConf
# from pyspark.context import SparkContext
# from pyspark.sql import HiveContext

# sc= SparkContext('local','example')
# hc = HiveContext(sc)
# tf1 = sc.textFile("hdfs://cdhstltest/user/data/demo.CSV")
# print(tf1.first())

# hc.sql("use intg_cme_w")
# spf = hc.sql("SELECT * FROM spf LIMIT 100")
# print(spf.show(5))


# Spark operations
There are two main types of Spark operations: 1. Transformations and 2. Actions 

[Link](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html#spark-operations)


# 1. Spark Transformations
![](./image/transforms.png)


# 2. Actions
![](./image/actions.png)

# rdd.DataFrame vs pd.DataFrame

## Create DataFrame
### From List

In [41]:
import pandas as pd
my_list = [['a', 1, 2], ['b', 2, 3], ['c', 3, 4]]
col_name = ['A', 'B', 'C']

# caution for the columns=
data = pd.DataFrame(my_list, columns=col_name)

data


,A,B,C
0,a,1,2
1,b,2,3
2,c,3,4


In [42]:
# Pay attentation to the parameter columns= in pd.DataFrame. Since the default value will make the list as rows.
pd.DataFrame(my_list, col_name)


,0,1,2
A,a,1,2
B,b,2,3
C,c,3,4


In [43]:
data = spark.createDataFrame(my_list, col_name)
data.show()


+---+---+---+
|  A|  B|  C|
+---+---+---+
|  a|  1|  2|
|  b|  2|  3|
|  c|  3|  4|
+---+---+---+



#### From Dict


In [44]:
d = {'A': [0, 1, 0],
     'B': [1, 0, 1],
     'C': [1, 0, 0]}

pd.DataFrame(d)


,A,B,C
0,0,1,1
1,1,0,0
2,0,1,0


In [45]:
import numpy as np
spark.createDataFrame(
    np.array(list(d.values())).T.tolist(), list(d.keys())).show()


+---+---+---+
|  A|  B|  C|
+---+---+---+
|  0|  1|  1|
|  1|  0|  0|
|  0|  1|  0|
+---+---+---+



### Load DataFrame

#### From csv
Most of time, you need to share your code with your colleagues or release your code for Code Review or Quality assurance(QA). You will definitely do not want to have your User Information in the code. So you can save them in login.txt:

In [48]:
# pd.DataFrame dp: DataFrame pandas
df = pd.read_csv('./data/Advertising.csv')
# rdd.DataFrame. dp: DataFrame spark
ds = spark.read.csv(path='./data/Advertising.csv',
                    sep=',',
                    encoding='UTF-8',
                    comment=None,
                    header=True,
                    inferSchema=True)


ds.show()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
| 66.1|  5.8|     24.2|  8.6|
|214.7| 24.0|      4.0| 17.4|
| 23.8| 35.1|     65.9|  9.2|
| 97.5|  7.6|      7.2|  9.7|
|204.1| 32.9|     46.0| 19.0|
|195.4| 47.7|     52.9| 22.4|
| 67.8| 36.6|    114.0| 12.5|
|281.4| 39.6|     55.8| 24.4|
| 69.2| 20.5|     18.3| 11.3|
|147.3| 23.9|     19.1| 14.6|
+-----+-----+---------+-----+
only showing top 20 rows



##  Pandas and Spark method

###  First n Rows

In [49]:
df.head(3)


,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3


In [50]:
ds.show(3)


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
+-----+-----+---------+-----+
only showing top 3 rows



### Column Names

In [51]:
df.columns, ds.columns


(Index(['TV', 'Radio', 'Newspaper', 'Sales'], dtype='object'),
 ['TV', 'Radio', 'Newspaper', 'Sales'])

### Data types

In [52]:
type(df), type(ds)


(pandas.core.frame.DataFrame, pyspark.sql.dataframe.DataFrame)

In [53]:
df.dtypes, ds.dtypes


(TV           float64
 Radio        float64
 Newspaper    float64
 Sales        float64
 dtype: object,
 [('TV', 'double'),
  ('Radio', 'double'),
  ('Newspaper', 'double'),
  ('Sales', 'double')])

### Fill Null

In [54]:
my_list = [['male', 1, None], ['female', 2, 3], ['male', 3, 4]]
df = pd.DataFrame(my_list, columns=['A', 'B', 'C'])
ds = spark.createDataFrame(my_list, ['A', 'B', 'C'])
#
df.head(), ds.show()


+------+---+----+
|     A|  B|   C|
+------+---+----+
|  male|  1|null|
|female|  2|   3|
|  male|  3|   4|
+------+---+----+



(        A  B    C
 0    male  1  NaN
 1  female  2  3.0
 2    male  3  4.0,
 None)

In [55]:
df.fillna(-99)


,A,B,C
0,male,1,-99.0
1,female,2,3.0
2,male,3,4.0


In [56]:
ds.fillna(-99).show()


+------+---+---+
|     A|  B|  C|
+------+---+---+
|  male|  1|-99|
|female|  2|  3|
|  male|  3|  4|
+------+---+---+



### Replace Values

In [57]:
# caution: you need to chose specific col
df.A.replace(['male', 'female'], [1, 0], inplace=True)
df


,A,B,C
0,1,1,NaN
1,0,2,3.0
2,1,3,4.0


In [58]:
# caution: Mixed type replacements are not supported
ds.na.replace(['male', 'female'], ['1', '0']).show()


+---+---+----+
|  A|  B|   C|
+---+---+----+
|  1|  1|null|
|  0|  2|   3|
|  1|  3|   4|
+---+---+----+



### Rename Columns

#### Rename all columns

In [60]:
dp = pd.read_csv('./data/Advertising.csv')
#
ds = spark.read.csv(path='./data/Advertising.csv',
                    header=True,
                    inferSchema=True)


dp.head()


,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [63]:
ds.columns


['TV', 'Radio', 'Newspaper', 'Sales']

In [61]:
dp.columns = ['a', 'b', 'c', 'd']
dp.head(4)


,a,b,c,d
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5


In [65]:
ds.toDF('a', 'b', 'c', 'd').show(4)


+-----+----+----+----+
|    a|   b|   c|   d|
+-----+----+----+----+
|230.1|37.8|69.2|22.1|
| 44.5|39.3|45.1|10.4|
| 17.2|45.9|69.3| 9.3|
|151.5|41.3|58.5|18.5|
+-----+----+----+----+
only showing top 4 rows



#### Rename one or more columns

In [66]:
mapping = {'a': 'TESR', 'c': 'D'}
dp.rename(columns=mapping).head(4)


,TESR,b,D,d
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5


In [67]:
new_names = [mapping.get(col, col) for col in ds.columns]
ds.toDF(*new_names).show(4)


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
+-----+-----+---------+-----+
only showing top 4 rows



You can also use withColumnRenamed to rename one column in PySpark.

In [68]:
ds.withColumnRenamed('B', 'Paper').show(4)


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
+-----+-----+---------+-----+
only showing top 4 rows



### Drop Columns

In [69]:
drop_name = ["d"]
print(dp.columns)
dp.drop(drop_name, axis=1).head(4)


Index(['a', 'b', 'c', 'd'], dtype='object')


,a,b,c
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5


In [70]:
drop_name = ['A']
print(ds.columns)
ds.drop(*drop_name).show(4)


['TV', 'Radio', 'Newspaper', 'Sales']
+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
+-----+-----+---------+-----+
only showing top 4 rows



### Filter

In [71]:
dp = pd.read_csv('./data/Advertising.csv')
#
ds = spark.read.csv(path='./data/Advertising.csv',
                    header=True,
                    inferSchema=True)


In [72]:
dp[dp.Newspaper < 20].head(4)


,TV,Radio,Newspaper,Sales
7,120.2,19.6,11.6,13.2
8,8.6,2.1,1.0,4.8
11,214.7,24.0,4.0,17.4
13,97.5,7.6,7.2,9.7


In [73]:
ds[ds.Newspaper < 20].show(4)


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|214.7| 24.0|      4.0| 17.4|
| 97.5|  7.6|      7.2|  9.7|
+-----+-----+---------+-----+
only showing top 4 rows



In [74]:
dp[(dp.Newspaper < 20) & (dp.TV > 100)].head(4)


,TV,Radio,Newspaper,Sales
7,120.2,19.6,11.6,13.2
11,214.7,24.0,4.0,17.4
19,147.3,23.9,19.1,14.6
25,262.9,3.5,19.5,12.0


In [75]:
#
ds[(ds.Newspaper < 20) & (ds.TV > 100)].show(3)


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|120.2| 19.6|     11.6| 13.2|
|214.7| 24.0|      4.0| 17.4|
|147.3| 23.9|     19.1| 14.6|
+-----+-----+---------+-----+
only showing top 3 rows



###  Shape 

In [76]:
dp.shape


(200, 4)

In [77]:
print((ds.count(), len(ds.columns)))


(200, 4)


### With New Column

In [78]:
dp['tv_norm'] = dp.TV/sum(dp.TV)
dp.head(4)


,TV,Radio,Newspaper,Sales,tv_norm
0,230.1,37.8,69.2,22.1,0.007824
1,44.5,39.3,45.1,10.4,0.001513
2,17.2,45.9,69.3,9.3,0.000585
3,151.5,41.3,58.5,18.5,0.005152


In [79]:
from pyspark.sql import functions as F

ds.withColumn('tv_norm', ds.TV /
              ds.groupBy().agg(F.sum("TV")).collect()[0][0]).show(4)


+-----+-----+---------+-----+--------------------+
|   TV|Radio|Newspaper|Sales|             tv_norm|
+-----+-----+---------+-----+--------------------+
|230.1| 37.8|     69.2| 22.1|0.007824268493802813|
| 44.5| 39.3|     45.1| 10.4|0.001513167961643...|
| 17.2| 45.9|     69.3|  9.3|5.848649200061207E-4|
|151.5| 41.3|     58.5| 18.5|0.005151571824472517|
+-----+-----+---------+-----+--------------------+
only showing top 4 rows



In [80]:
dp['log_tv'] = np.log(dp.TV)
dp.head(4)


,TV,Radio,Newspaper,Sales,tv_norm,log_tv
0,230.1,37.8,69.2,22.1,0.007824,5.438514
1,44.5,39.3,45.1,10.4,0.001513,3.795489
2,17.2,45.9,69.3,9.3,0.000585,2.844909
3,151.5,41.3,58.5,18.5,0.005152,5.020586


In [81]:
#
import pyspark.sql.functions as F
ds.withColumn('log_tv', F.log(ds.TV)).show(4)


+-----+-----+---------+-----+------------------+
|   TV|Radio|Newspaper|Sales|            log_tv|
+-----+-----+---------+-----+------------------+
|230.1| 37.8|     69.2| 22.1|  5.43851399704132|
| 44.5| 39.3|     45.1| 10.4|3.7954891891721947|
| 17.2| 45.9|     69.3|  9.3|2.8449093838194073|
|151.5| 41.3|     58.5| 18.5| 5.020585624949424|
+-----+-----+---------+-----+------------------+
only showing top 4 rows



In [82]:
dp['tv+10'] = dp.TV.apply(lambda x: x+10)
dp.head(4)


,TV,Radio,Newspaper,Sales,tv_norm,log_tv,tv+10
0,230.1,37.8,69.2,22.1,0.007824,5.438514,240.1
1,44.5,39.3,45.1,10.4,0.001513,3.795489,54.5
2,17.2,45.9,69.3,9.3,0.000585,2.844909,27.2
3,151.5,41.3,58.5,18.5,0.005152,5.020586,161.5


In [83]:
ds.withColumn('tv+10', ds.TV+10).show(4)


+-----+-----+---------+-----+-----+
|   TV|Radio|Newspaper|Sales|tv+10|
+-----+-----+---------+-----+-----+
|230.1| 37.8|     69.2| 22.1|240.1|
| 44.5| 39.3|     45.1| 10.4| 54.5|
| 17.2| 45.9|     69.3|  9.3| 27.2|
|151.5| 41.3|     58.5| 18.5|161.5|
+-----+-----+---------+-----+-----+
only showing top 4 rows



## Join

![](./image/hMKKt.jpg)

In [84]:
leftp = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=[0, 1, 2, 3])

rightp = pd.DataFrame({'A': ['A0', 'A1', 'A6', 'A7'],
                       'F': ['B4', 'B5', 'B6', 'B7'],
                       'G': ['C4', 'C5', 'C6', 'C7'],
                       'H': ['D4', 'D5', 'D6', 'D7']},
                      index=[4, 5, 6, 7])

lefts = spark.createDataFrame(leftp)
rights = spark.createDataFrame(rightp)
lefts.show(), rights.show()


+---+---+---+---+
|  A|  B|  C|  D|
+---+---+---+---+
| A0| B0| C0| D0|
| A1| B1| C1| D1|
| A2| B2| C2| D2|
| A3| B3| C3| D3|
+---+---+---+---+

+---+---+---+---+
|  A|  F|  G|  H|
+---+---+---+---+
| A0| B4| C4| D4|
| A1| B5| C5| D5|
| A6| B6| C6| D6|
| A7| B7| C7| D7|
+---+---+---+---+



(None, None)

### Left Join

In [85]:
leftp.merge(rightp, on='A', how='left')


,A,B,C,D,F,G,H
0,A0,B0,C0,D0,B4,C4,D4
1,A1,B1,C1,D1,B5,C5,D5
2,A2,B2,C2,D2,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN


In [86]:
lefts.join(rights, on='A', how='left').orderBy('A', ascending=True).show()


+---+---+---+---+----+----+----+
|  A|  B|  C|  D|   F|   G|   H|
+---+---+---+---+----+----+----+
| A0| B0| C0| D0|  B4|  C4|  D4|
| A1| B1| C1| D1|  B5|  C5|  D5|
| A2| B2| C2| D2|null|null|null|
| A3| B3| C3| D3|null|null|null|
+---+---+---+---+----+----+----+



### Right Join


In [87]:
leftp.merge(rightp, on='A', how='right')


,A,B,C,D,F,G,H
0,A0,B0,C0,D0,B4,C4,D4
1,A1,B1,C1,D1,B5,C5,D5
2,A6,NaN,NaN,NaN,B6,C6,D6
3,A7,NaN,NaN,NaN,B7,C7,D7


In [88]:
lefts.join(rights, on='A', how='right').orderBy('A', ascending=True).show()


+---+----+----+----+---+---+---+
|  A|   B|   C|   D|  F|  G|  H|
+---+----+----+----+---+---+---+
| A0|  B0|  C0|  D0| B4| C4| D4|
| A1|  B1|  C1|  D1| B5| C5| D5|
| A6|null|null|null| B6| C6| D6|
| A7|null|null|null| B7| C7| D7|
+---+----+----+----+---+---+---+



### Inner Join




In [89]:
leftp.merge(rightp, on='A', how='inner')


,A,B,C,D,F,G,H
0,A0,B0,C0,D0,B4,C4,D4
1,A1,B1,C1,D1,B5,C5,D5


In [90]:
lefts.join(rights, on='A', how='inner').orderBy('A', ascending=True).show()


+---+---+---+---+---+---+---+
|  A|  B|  C|  D|  F|  G|  H|
+---+---+---+---+---+---+---+
| A0| B0| C0| D0| B4| C4| D4|
| A1| B1| C1| D1| B5| C5| D5|
+---+---+---+---+---+---+---+



### Full Join



In [91]:
leftp.merge(rightp, on='A', how='outer')


,A,B,C,D,F,G,H
0,A0,B0,C0,D0,B4,C4,D4
1,A1,B1,C1,D1,B5,C5,D5
2,A2,B2,C2,D2,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN
4,A6,NaN,NaN,NaN,B6,C6,D6
5,A7,NaN,NaN,NaN,B7,C7,D7


In [92]:
lefts.join(rights, on='A', how='full').orderBy('A', ascending=True).show()


+---+----+----+----+----+----+----+
|  A|   B|   C|   D|   F|   G|   H|
+---+----+----+----+----+----+----+
| A0|  B0|  C0|  D0|  B4|  C4|  D4|
| A1|  B1|  C1|  D1|  B5|  C5|  D5|
| A2|  B2|  C2|  D2|null|null|null|
| A3|  B3|  C3|  D3|null|null|null|
| A6|null|null|null|  B6|  C6|  D6|
| A7|null|null|null|  B7|  C7|  D7|
+---+----+----+----+----+----+----+



### Concat Columns

![](./image/merging_append3.png)

In [93]:
my_list = [('a', 2, 3),
           ('b', 5, 6),
           ('c', 8, 9),
           ('a', 2, 3),
           ('b', 5, 6),
           ('c', 8, 9)]
col_name = ['col1', 'col2', 'col3']
#
dp = pd.DataFrame(my_list, columns=col_name)
ds = spark.createDataFrame(my_list, schema=col_name)


In [94]:
dp['concat'] = dp.apply(lambda x: '%s%s' % (x['col1'], x['col2']), axis=1)
dp


,col1,col2,col3,concat
0,a,2,3,a2
1,b,5,6,b5
2,c,8,9,c8
3,a,2,3,a2
4,b,5,6,b5
5,c,8,9,c8


In [95]:
ds.withColumn('concat', F.concat('col1', 'col2')).show()


+----+----+----+------+
|col1|col2|col3|concat|
+----+----+----+------+
|   a|   2|   3|    a2|
|   b|   5|   6|    b5|
|   c|   8|   9|    c8|
|   a|   2|   3|    a2|
|   b|   5|   6|    b5|
|   c|   8|   9|    c8|
+----+----+----+------+



## GroupBy

![](./image/transform-example.png)

In [96]:
dp.groupby(['col1']).agg({'col2': 'min', 'col3': 'mean'})


,col2,col3
col1,,
a,2,3.0
b,5,6.0
c,8,9.0


In [97]:
ds.groupBy(['col1']).agg({'col2': 'min', 'col3': 'avg'}).show()


+----+---------+---------+
|col1|min(col2)|avg(col3)|
+----+---------+---------+
|   c|        8|      9.0|
|   b|        5|      6.0|
|   a|        2|      3.0|
+----+---------+---------+



## Pivot

https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

![](./image/reshaping_pivot.png)

In [98]:
pd.pivot_table(dp, values='col3', index='col1', columns='col2', aggfunc=np.sum)


col2,2,5,8
col1,,,
a,6.0,NaN,NaN
b,NaN,12.0,NaN
c,NaN,NaN,18.0


In [99]:
ds.groupBy(['col1']).pivot('col2').sum('col3').show()


+----+----+----+----+
|col1|   2|   5|   8|
+----+----+----+----+
|   c|null|null|  18|
|   b|null|  12|null|
|   a|   6|null|null|
+----+----+----+----+



##  Window
![](./image/Window-Operations-01-2.jpg)

In [100]:
d = {'A': ['a', 'b', 'c', 'd'], 'B': ['m', 'm', 'n', 'n'], 'C': [1, 2, 3, 6]}
dp = pd.DataFrame(d)
ds = spark.createDataFrame(dp)


In [101]:
dp['rank'] = dp.groupby('B')['C'].rank('dense', ascending=False)
dp


,A,B,C,rank
0,a,m,1,2.0
1,b,m,2,1.0
2,c,n,3,2.0
3,d,n,6,1.0


In [102]:
from pyspark.sql.window import Window
w = Window.partitionBy('B').orderBy(ds.C.desc())
ds = ds.withColumn('rank', F.rank().over(w))
ds.show()


+---+---+---+----+
|  A|  B|  C|rank|
+---+---+---+----+
|  b|  m|  2|   1|
|  a|  m|  1|   2|
|  d|  n|  6|   1|
|  c|  n|  3|   2|
+---+---+---+----+



## Rank vs Dense_rank

In [103]:
d = {'Id': [1, 2, 3, 4, 5, 6],
     'Score': [4.00, 4.00, 3.85, 3.65, 3.65, 3.50]}
#
data = pd.DataFrame(d)
dp = data.copy()
dp


,Id,Score
0,1,4.00
1,2,4.00
2,3,3.85
3,4,3.65
4,5,3.65
5,6,3.50


In [104]:
dp['Rank_dense'] = dp['Score'].rank(method='dense', ascending=False)
dp['Rank'] = dp['Score'].rank(method='min', ascending=False)
dp


,Id,Score,Rank_dense,Rank
0,1,4.00,1.0,1.0
1,2,4.00,1.0,1.0
2,3,3.85,2.0,3.0
3,4,3.65,3.0,4.0
4,5,3.65,3.0,4.0
5,6,3.50,4.0,6.0


In [105]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
ds = spark.createDataFrame(data)


w = Window.orderBy(ds.Score.desc())
ds = ds.withColumn('Rank_spark_dense', F.dense_rank().over(w))
ds = ds.withColumn('Rank_spark', F.rank().over(w))
ds.show()


+---+-----+----------------+----------+
| Id|Score|Rank_spark_dense|Rank_spark|
+---+-----+----------------+----------+
|  1|  4.0|               1|         1|
|  2|  4.0|               1|         1|
|  3| 3.85|               2|         3|
|  4| 3.65|               3|         4|
|  5| 3.65|               3|         4|
|  6|  3.5|               4|         6|
+---+-----+----------------+----------+



22/09/29 22:31:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
